In [10]:
import networkx as nx
from gurobipy import Model, GRB

# ================================
# 1. CREAZIONE GRAFO
# ================================
G = nx.DiGraph()
G.add_node("Depot1", type="depot", charging_possible=True)
G.add_node("Depot2", type="depot", charging_possible=True)
G.add_node("Stop1", type="stop", charging_possible=True)
G.add_node("Stop2", type="stop", charging_possible=False)
G.add_node("Stop3", type="stop", charging_possible=True)
G.add_node("Stop4", type="stop", charging_possible=False)
G.add_node("Stop5", type="stop", charging_possible=True)



G.add_edge("Depot1", "Stop1", distance=3)
G.add_edge("Stop1", "Stop2", distance=4)
G.add_edge("Stop2", "Stop3", distance=2)
G.add_edge("Stop3", "Stop5", distance=5)
G.add_edge("Stop5", "Depot2", distance=4)
G.add_edge("Depot1", "Stop4", distance=6)
G.add_edge("Stop4", "Stop5", distance=7)
G.add_edge("Depot2", "Stop3", distance=3)

C = ["c1"]  # set of c-type plugs devices
N = list(G.nodes) # feasible charging stop set
D = [n for n, attr in G.nodes(data=True) if attr.get('type') == 'depot']

print(N)

['Depot1', 'Depot2', 'Stop1', 'Stop2', 'Stop3', 'Stop4', 'Stop5']


In [1]:
import gurobipy as gb
import math

nv_rb_0 = {
    "r1" : {"M103": 3, "M104": 2},
    "r2" : {"M103": 2, "M104": 0},
    "r3" : {"M103": 1, "M104": 2},
    "r4" : {"M103": 1, "M104": 1},
}  # number of b-type *non-battery* buses on route r

nob_rb = {
    "r1": {"E433": 0},
    "r2": {"E433": 1},
    "r3": {"E433": 2},
    "r4": {"E433": 2},
}  # number of *old b-type electric* buses on route r


V = ["M103", "M104"] # non battery vehicle type set
R = ["r1","r2","r3","r4"] # route set
B = ["E433", "E420", "E302"] #[E433, E420, E321, E490, 321D, 420D] # electric bus-type

B_r = {
    "r1": ["E433", "E420", "E302"],
    "r2": ["E433"],         
    "r3": ["E420", "E302"],
    "r4": ["E433", "E420"],
} # electric bus type set of route r

capacities = [153, 87, 175, 130, 80] #starting from electric and then non battery vehicles

cap_b = {node: cap for node, cap in zip(B + V, capacities)}

dem_r = {}  # passenger demand of route r = past passenger capacity of all route r vehicles
            
# Calculate the passenger demand for each route
for r in set(nv_rb_0.keys()).union(nob_rb.keys()):
    dem = 0
    # Add capacities from non-battery buses
    for b, n in nv_rb_0.get(r, {}).items():
        dem += cap_b.get(b, 0) * n

    # Add capacities from old electric buses
    for b, n in nob_rb.get(r, {}).items():
        dem += cap_b.get(b, 0) * n

    dem_r[r] = dem

dem_0_r = {} # passenger capacity of route r to be satisfied by new electric buses and remaining non-battery vehicles
for r in R:
    dem_0_r[r] = dem_r[r] - sum(nob_rb[r].get(b, 0) * cap_b[b] for b in B_r[r])  ## calculating dem_0_r!
    # .get used because if we don't find a "bus" we just have 0 and not a crash (like with nob_rb[r][b])
    # no need of quicksum becuse we have only inputs and no variables

ub_rb = {
    # (1, 'busA'): 3,
} # upper bound on the number of new b-type electric buses
for r in R:
    for b in B_r[r]: # assuming B_r[r] gives buses relevant to route r            ## calculating ub_rb
        numerator = dem_0_r[r]
        denominator = cap_b[b]
        ub_rb[r, b] = math.ceil(numerator/denominator)

print(ub_rb)

{('r1', 'E433'): 4, ('r1', 'E420'): 7, ('r1', 'E302'): 4, ('r2', 'E433'): 2, ('r3', 'E420'): 7, ('r3', 'E302'): 4, ('r4', 'E433'): 2, ('r4', 'E420'): 3}


In [43]:
import data_inizialization as di
import numpy as np

R = ["r1","r2","r3","r4"] # route set
B = ["E433", "E420", "E302"] #[E433, E420, E321, E490, 321D, 420D] # electric bus-type
C = ["c1"] # charging type set   # In the base case |C| = 1 -> c = 1 -> we just have one charging type -> In the random cases, so modified base cases -> we several c types

n_rbc_data_2d = np.array([
    [1, 1, 1, 1],  # E433
    [2, 1, 1, 2],  # E420
    [2, 2, 2, 2],  # E302
])

n_rbc_data = n_rbc_data_2d[:, :, np.newaxis] #just this case since we need also a c dimensione even if it is just 1

n_rbc = di.init_n_rbc(n_rbc_data, R, B, C) # Initialize n_rbc with data from data_inizialization module

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
import data_inizialization as di
import numpy as np
import importlib

importlib.reload(di)

R = ["r1","r2","r3","r4"] # route set
B = ["E433", "E420", "E302"] #[E433, E420, E321, E490, 321D, 420D] # electric bus-type
C = ["c1"] # charging type set   # In the base case |C| = 1 -> c = 1 -> we just have one charging type -> In the random cases, so modified base cases -> we several c types

n_rbc_data_2d = np.array([
    [1, 1, 1, 1],  # E433
    [2, 1, 0, 2],  # E420
    [2, 2, 2, 2],  # E302
])

n_rbc_data = n_rbc_data_2d[:, :, np.newaxis].transpose(1, 0, 2) #just this case since we need also a c dimensione even if it is just 1

n_rbc = di.init_n_rbc(n_rbc_data, R, B, C) # Initialize n_rbc with data from data_inizialization module

print(n_rbc)

{('r1', 'E433', 'c1'): 1, ('r1', 'E420', 'c1'): 2, ('r1', 'E302', 'c1'): 2, ('r2', 'E433', 'c1'): 1, ('r2', 'E420', 'c1'): 1, ('r2', 'E302', 'c1'): 2, ('r3', 'E433', 'c1'): 1, ('r3', 'E420', 'c1'): 0, ('r3', 'E302', 'c1'): 2, ('r4', 'E433', 'c1'): 1, ('r4', 'E420', 'c1'): 2, ('r4', 'E302', 'c1'): 2}


In [ ]:
R = ["r1","r2","r3","r4"] # route set
B_r = {
    "r1": ["E433", "E420", "E302"],
    "r2": ["E433"],         
    "r3": ["E420", "E302"],
    "r4": ["E433", "E420"],
} # electric bus type set of route r

for r in R:
    for b in range(len(B_r[r])):
        print()




0
1
2
3
